In [1]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen
from bs4 import BeautifulSoup
import io
import datetime
import matplotlib.dates as mdates

In [2]:
# Data location
from sys import platform
if platform == "linux" or platform == "linux2":
    data_string = "/home/maria/Documents/data/pied_piper/green/"
    output_string = "/home/maria/Documents/output/pied_piper/green/"

elif platform == "win32"or platform == "win64":
    data_string = "C:/Users/maria/OneDrive/Documents/data/pied_piper/green/"
    output_string = "C:/Users/maria/OneDrive/Documents/output/pied_piper/green/"
else:
    data_string = "/Users/erikamiller/Desktop/Pied Piper/Data/Green/"
    output_string = "/Users/erikamiller/Desktop/Pied Piper/Output/Green/"

In [3]:
# Read csv and assign it to df
year = 2014
csv_name = data_string+str(year)+"_green_all_R_new.csv"
df = pd.read_csv(csv_name, header=0)

In [4]:
# Format date & time
df['midpoint'] = pd.to_datetime(df['midpoint'],errors='coerce')
df['start_datetime'] = pd.to_datetime(df['start_datetime'], errors='coerce')
df['end_datetime'] = pd.to_datetime(df['end_datetime'], errors='coerce')

df['Date'] = pd.to_datetime(df['start_datetime']).dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['doy'] = df['Date'].dt.dayofyear

In [5]:
# Checking whether the midpoint of the time when trap was in during day or night
def categorize_daytime(dt):
    if dt.hour >= 6 and dt.hour < 18:
        return 'Day'
    else:
        return 'Night'
df['daytime_category'] = df['midpoint'].apply(categorize_daytime)

In [6]:
# List the columns of the dataframe
list(df)

['Unnamed: 0',
 'TrapSiteLocation',
 'StartDate',
 'StartTime',
 'EndDate',
 'EndTime',
 'TotalCatch',
 'Brook Lamprey/NA/Mixed/WILD',
 'chinook0_hatchery_num',
 'chinook0_wild_num',
 'chinook1_hatchery_num',
 'chinook1_wild_num',
 'chum0_mixed_num',
 'coho0_wild_num',
 'coho1_hatchery_num',
 'coho1_wild_num',
 'Cutthroat/NA/Adult/WILD',
 'Cutthroat/NA/Parr - Unknown/WILD',
 'Cutthroat/NA/Smolt Yearling/WILD',
 'pink0_wild',
 'River Lamprey/NA/Mixed/WILD',
 'Steelhead/Summer-Winter/Parr - Unknown/WILD',
 'steelheadsmolt_hatchery_num',
 'steelheadsmolt_wild_num',
 'Trout-General/NA/Parr - Unknown/WILD',
 'start_datetime',
 'start_time',
 'end_datetime',
 'In',
 'midpoint',
 'Date',
 'doy',
 'daytime_category']

In [7]:
# Drop columns from the dataframe
df.drop(['TrapSiteLocation', 'StartTime', 'EndTime', 'TotalCatch', 'Brook Lamprey/NA/Mixed/WILD', 'Cutthroat/NA/Adult/WILD',
         'Cutthroat/NA/Parr - Unknown/WILD', 'Cutthroat/NA/Smolt Yearling/WILD','River Lamprey/NA/Mixed/WILD',
         'Steelhead/Summer-Winter/Parr - Unknown/WILD', 'Trout-General/NA/Parr - Unknown/WILD', 'start_time'], axis=1, inplace=True)

In [9]:
# Calculate CPUE (Catch Per Unit Effort)

df['chinook0_hatchery_perhour'] = df['chinook0_hatchery_num']/df['In']
df['chinook0_wild_perhour'] = df['chinook0_wild_num']/df['In']

df['chinook1_hatchery_perhour'] = df['chinook1_hatchery_num']/df['In']
df['chinook1_wild_perhour'] = df['chinook1_wild_num']/df['In']

df['chum0_mixed_perhour'] = df['chum0_mixed_num']/df['In']

df['coho0_wild_perhour'] = df['coho0_wild_num']/df['In']

df['coho1_hatchery_perhour'] = df['coho1_hatchery_num']/df['In']
df['coho1_wild_perhour'] = df['coho1_wild_num']/df['In']
# df['coho1_mixed_perhour'] = df['coho1_mixed_num']/df['In']

df['pink0_wild_perhour'] = df['pink0_wild']/df['In']

df['steelheadsmolt_hatchery_perhour'] = df['steelheadsmolt_hatchery_num']/df['In']
df['steelheadsmolt_wild_perhour'] = df['steelheadsmolt_wild_num']/df['In']

In [10]:
# Make 2 separate dataframe categories (Day & Night)
df_day = df[df['daytime_category']=="Day"]
df_night = df[df['daytime_category']=="Night"]

In [11]:
# Check for NaN or missing values in the 'midpoint' column of df_day and df_night
df_day[df_day['midpoint'].isnull()]
df_night[df_night['midpoint'].isnull()]

## May not need

,Unnamed: 0,StartDate,EndDate,chinook0_hatchery_num,chinook0_wild_num,chinook1_hatchery_num,chinook1_wild_num,chum0_mixed_num,coho0_wild_num,coho1_hatchery_num,...,chinook0_wild_perhour,chinook1_hatchery_perhour,chinook1_wild_perhour,chum0_mixed_perhour,coho0_wild_perhour,coho1_hatchery_perhour,coho1_wild_perhour,pink0_wild_perhour,steelheadsmolt_hatchery_perhour,steelheadsmolt_wild_perhour


In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #display(df)
    #display(df_day)
    display(df_night) 

,Unnamed: 0,StartDate,EndDate,chinook0_hatchery_num,chinook0_wild_num,chinook1_hatchery_num,chinook1_wild_num,chum0_mixed_num,coho0_wild_num,coho1_hatchery_num,coho1_wild_num,pink0_wild,steelheadsmolt_hatchery_num,steelheadsmolt_wild_num,start_datetime,end_datetime,In,midpoint,Date,doy,daytime_category,chinook0_hatchery_perhour,chinook0_wild_perhour,chinook1_hatchery_perhour,chinook1_wild_perhour,chum0_mixed_perhour,coho0_wild_perhour,coho1_hatchery_perhour,coho1_wild_perhour,pink0_wild_perhour,steelheadsmolt_hatchery_perhour,steelheadsmolt_wild_perhour
0,1,2014-01-24,2014-01-25,0,11,0,0,0,0,0,2,8,0,0,2014-01-24 16:00:00,2014-01-25 08:00:00,16.000000,2014-01-25 00:00:00,2014-01-24,24,Night,0.000000,0.687500,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.500000,0.000000,0.000000
2,3,2014-01-25,2014-01-26,0,2,0,0,0,0,0,5,3,0,0,2014-01-25 17:00:00,2014-01-26 08:00:00,15.000000,2014-01-26 00:30:00,2014-01-25,25,Night,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.200000,0.000000,0.000000
4,5,2014-01-26,2014-01-27,0,6,0,0,0,0,0,3,2,0,0,2014-01-26 17:00:00,2014-01-27 08:30:00,15.500000,2014-01-27 00:45:00,2014-01-26,26,Night,0.000000,0.387097,0.000000,0.000000,0.000000,0.000000,0.000000,0.193548,0.129032,0.000000,0.000000
6,7,2014-01-27,2014-01-28,0,4,0,0,0,0,0,3,0,0,0,2014-01-27 16:45:00,2014-01-28 08:00:00,15.250000,2014-01-28 00:22:30,2014-01-27,27,Night,0.000000,0.262295,0.000000,0.000000,0.000000,0.000000,0.000000,0.196721,0.000000,0.000000,0.000000
8,9,2014-01-28,2014-01-29,0,24,0,4,0,14,0,0,0,0,1,2014-01-28 16:45:00,2014-01-29 08:30:00,15.750000,2014-01-29 00:37:30,2014-01-28,28,Night,0.000000,1.523810,0.000000,0.253968,0.000000,0.888889,0.000000,0.000000,0.000000,0.000000,0.063492
10,11,2014-01-29,2014-01-30,0,219,0,0,0,0,0,10,124,0,2,2014-01-29 16:30:00,2014-01-30 09:00:00,16.500000,2014-01-30 00:45:00,2014-01-29,29,Night,0.000000,13.272727,0.000000,0.000000,0.000000,0.000000,0.000000,0.606061,7.515152,0.000000,0.121212
12,13,2014-01-30,2014-01-31,0,41,0,0,0,0,0,5,8,0,0,2014-01-30 16:45:00,2014-01-31 08:45:00,16.000000,2014-01-31 00:45:00,2014-01-30,30,Night,0.000000,2.562500,0.000000,0.000000,0.000000,0.000000,0.000000,0.312500,0.500000,0.000000,0.000000
14,15,2014-01-31,2014-02-01,0,21,0,0,0,0,0,4,11,0,0,2014-01-31 17:00:00,2014-02-01 08:00:00,15.000000,2014-02-01 00:30:00,2014-01-31,31,Night,0.000000,1.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.266667,0.733333,0.000000,0.000000
16,17,2014-02-01,2014-02-02,0,19,0,0,0,0,0,4,9,0,0,2014-02-01 17:00:00,2014-02-02 08:00:00,15.000000,2014-02-02 00:30:00,2014-02-01,32,Night,0.000000,1.266667,0.000000,0.000000,0.000000,0.000000,0.000000,0.266667,0.600000,0.000000,0.000000
17,18,2014-02-02,2014-02-03,0,4,0,0,0,0,0,2,3,0,0,2014-02-02 08:00:00,2014-02-03 08:45:00,24.750000,2014-02-02 20:22:30,2014-02-02,33,Night,0.000000,0.161616,0.000000,0.000000,0.000000,0.000000,0.000000,0.080808,0.121212,0.000000,0.000000


In [13]:
# Need to have one row for every date
df_dates_day = pd.DataFrame({'date_range' : 
                         pd.date_range(start='2014-01-24 12:00:00', 
                                       end='2014-07-22 12:00:00')})
df_dates_night = pd.DataFrame({'date_range' : 
                         pd.date_range(start='2014-01-24 00:00:00', 
                                       end='2014-07-22 00:00:00')})

df_merged_day = pd.merge_asof(df_dates_day, 
              df_day, left_on = 'date_range', 
              right_on = 'midpoint', direction = 'nearest',
              tolerance = pd.Timedelta(hours = 12))

df_merged_night = pd.merge_asof(df_dates_night, 
              df_night, left_on = 'date_range', 
              right_on = 'midpoint', direction = 'nearest',
              tolerance = pd.Timedelta(hours = 12))

In [14]:
# doy columns cannot be nan, so making those columns again
# I will keep the current doy in midpoint doy 
# daytime_category columns cannot be nan

df_merged_day['midpoint_doy'] = df_merged_day['doy']
df_merged_day['doy'] = df_merged_day['date_range'].dt.dayofyear
df_merged_day['year'] = df_merged_day['date_range'].dt.year
df_merged_day['daytime_category'] = 'day'

df_merged_night['midpoint_doy'] = df_merged_night['doy']
df_merged_night['doy'] = df_merged_night['date_range'].dt.dayofyear
df_merged_night['year'] = df_merged_night['date_range'].dt.year
df_merged_night['daytime_category'] = 'night'

In [15]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #display(df_merged_night)
    display(df_merged_day)

,date_range,Unnamed: 0,StartDate,EndDate,chinook0_hatchery_num,chinook0_wild_num,chinook1_hatchery_num,chinook1_wild_num,chum0_mixed_num,coho0_wild_num,coho1_hatchery_num,coho1_wild_num,pink0_wild,steelheadsmolt_hatchery_num,steelheadsmolt_wild_num,start_datetime,end_datetime,In,midpoint,Date,doy,daytime_category,chinook0_hatchery_perhour,chinook0_wild_perhour,chinook1_hatchery_perhour,chinook1_wild_perhour,chum0_mixed_perhour,coho0_wild_perhour,coho1_hatchery_perhour,coho1_wild_perhour,pink0_wild_perhour,steelheadsmolt_hatchery_perhour,steelheadsmolt_wild_perhour,midpoint_doy,year
0,2014-01-24 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,24,day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014
1,2014-01-25 12:00:00,2.0,2014-01-25,2014-01-25,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014-01-25 08:00:00,2014-01-25 17:00:00,9.000000,2014-01-25 12:30:00,2014-01-25,25,day,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.0,2014
2,2014-01-26 12:00:00,4.0,2014-01-26,2014-01-26,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014-01-26 08:00:00,2014-01-26 17:00:00,9.000000,2014-01-26 12:30:00,2014-01-26,26,day,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.0,2014
3,2014-01-27 12:00:00,6.0,2014-01-27,2014-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014-01-27 08:30:00,2014-01-27 16:45:00,8.250000,2014-01-27 12:37:30,2014-01-27,27,day,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27.0,2014
4,2014-01-28 12:00:00,8.0,2014-01-28,2014-01-28,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2014-01-28 08:00:00,2014-01-28 16:45:00,8.750000,2014-01-28 12:22:30,2014-01-28,28,day,0.000000,0.571429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.114286,0.000000,0.000000,28.0,2014
5,2014-01-29 12:00:00,10.0,2014-01-29,2014-01-29,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,2014-01-29 08:30:00,2014-01-29 16:30:00,8.000000,2014-01-29 12:30:00,2014-01-29,29,day,0.000000,2.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.125000,0.000000,0.000000,29.0,2014
6,2014-01-30 12:00:00,12.0,2014-01-30,2014-01-30,0.0,22.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,0.0,0.0,2014-01-30 09:00:00,2014-01-30 16:45:00,7.750000,2014-01-30 12:52:30,2014-01-30,30,day,0.000000,2.838710,0.000000,0.000000,0.000000,0.000000,0.000000,0.129032,0.903226,0.000000,0.000000,30.0,2014
7,2014-01-31 12:00:00,14.0,2014-01-31,2014-01-31,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014-01-31 08:45:00,2014-01-31 17:00:00,8.250000,2014-01-31 12:52:30,2014-01-31,31,day,0.000000,1.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.0,2014
8,2014-02-01 12:00:00,16.0,2014-02-01,2014-02-01,0.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2014-02-01 08:00:00,2014-02-01 17:00:00,9.000000,2014-02-01 12:30:00,2014-02-01,32,day,0.000000,0.555556,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,32.0,2014
9,2014-02-02 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,33,day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014


In [17]:
# Save csv
df_merged_day.to_csv(data_string + "green_2014_day.csv")
df_merged_night.to_csv(data_string + "green_2014_night.csv")